In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import glob
import os
from IPython.display import display
import matplotlib.pyplot as plt
import json
from sklearn import preprocessing
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from keras.preprocessing.text import Tokenizer
import tensorflow_addons as tfa
import decimal
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler

In [2]:
trailers_csv = "/Volumes/Seagate/natasha-diploma/trailers.csv"
trailers_df = pd.read_csv(trailers_csv, index_col=None, header=0)
trailers_df.drop(trailers_df.columns[trailers_df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
display(trailers_df.head())

,title_x,duration,release_date,restrict,orig_title,kinopoisk_id,imdb_rating,kinopoisk_rating,synopsis,categories,...,emotions,Place of act,Qualities,Time of act,Based on,Audience,Mood,Subgenre,About,Theme
0,Философия Фила,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,"Фил – стоматолог, страдающий от депрессии. Его...",{Фильмы},...,"{'neutral': 0.5312193632125854, 'negative': 0....",['США'],['дебютный'],"['21-й век', 'новейшее время']",['оригинального сценария'],"['для молодёжи', 'для взрослых']","['грустный', 'депрессивный', 'трогательный']","['Драмы', 'Комедийные']","['преследования', 'самозванцы', 'расследование...","['развлекательные', 'психологический']"
1,Зубная фея,5123.0,2019-04-02,18.0,Tooth Fairy,1126022,2.2,0.00,Старая женщина начинает рассказывать легенду о...,{Фильмы},...,"{'negative': 0.4688006341457367, 'skip': 0.392...",['не определено'],"['зрелищные', 'жанровый', 'неожиданный финал']",['не определено'],"['эпоса и легенд', 'оригинального сценария']","['для взрослых', 'для мужчин', 'для подготовле...","['трагичные', 'мрачные', 'шокирующие', 'захват...","['ужасы', 'триллер']","['преследования', 'страхи', 'дети', 'молодежь'...",['развлекательные']
2,Проснись,4667.0,2019-01-19,16.0,Wake Up,1069713,2.9,0.00,"Врач-психиатр исследует дневник девушки, котор...",{Фильмы},...,"{'positive': 0.3140605390071869, 'negative': 0...",['не определено'],"['жанровый', 'дебютный']",['новейшее время'],['оригинального сценария'],"['для взрослых', 'для подготовленного зрителя'...","['захватывающие', 'жестокие', 'напряженные', '...","['Драмы', 'Боевики', 'триллер']","['психотерапевты', 'кровь', 'тайна смерти', 'п...","['психологический', 'развлекательные']"
3,[4k] Адская кухня,6148.0,2019-08-09,16.0,The Kitchen,1117988,5.5,5.50,"Смотрите в 4К! Три жены ирландских гангстеров,...",{Фильмы},...,"{'neutral': 0.4765896201133728, 'negative': 0....","['большой город', 'США']","['жанровый', 'остросюжетные']",['20-й век'],['комиксов'],"['для мужчин', 'для женщин', 'для взрослых']","['захватывающие', 'авантюрный', 'жестокие', 'н...","['Боевики', 'Драмы', 'криминальные']","['враги или вражда', 'женщины', 'антигерои', '...","['авантюрные', 'гангстерская']"
4,[4k] Покемон. Детектив Пикачу,6013.0,2019-05-02,12.0,Pokémon Detective Pikachu,994864,6.6,6.56,Смотрите в 4K! Элитный детектив Пикачу отправл...,"{Для детей,Фильмы}",...,"{'neutral': 0.4532718360424042, 'negative': 0....","['вымышленная страна (город)', 'большой город']","['частично анимационный', 'зрелищные', 'кассов...",['21-й век'],"['оригинального сценария', 'сериалов']","['для детей 13-16 лет', '9 лет', '8 лет', '7-1...","['авантюрный', 'захватывающие', 'динамичный', ...","['детские', 'Приключенческие', 'Фантастические...","['притяжение противоположностей', 'риск', 'пре...","['развлекательные', 'образовательный']"


In [3]:
sessions_csv = "/Volumes/Seagate/natasha-diploma/sessions.csv"
sessions_df = pd.read_csv(sessions_csv, index_col=None, header=0)
sessions_df.drop(sessions_df.columns[sessions_df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
display(sessions_df)

,0,1,2,3,4,5,6,7,8,9
0,403a42b604f504069c41d39c5a4fc2,23ee5640ec588ffaa78afa3268b04a,5b150a1cae009459493bd0fe5c7b3e,e291f8ae1de4a38c71027e9c91d51c,34cfd44ad5cc2d63441ebf437eccb5,02fbcf93a730c9c9d5e65aef41c67e,26b3f811a6b9ed75d3cc5198105fa4,6fcff414343456465369b05f6851eb,e2f15819192cded77ed1825a129c0c,41b772a5b18bc724562bfb60f4f3ac
1,a5b2dd3ed41aabb5733c7cfffff0bc,25e573b81ab69d6cf700a51281fc2a,49c58f679f72e0f02b97bdeee1a1bb,c0be492c14bcec5ab3dfd34d95d6f5,b2274b9b4c8c4dc2f79d417ddd71e5,7fd08607f52b89a753437eee177e32,25d41576f3529994c878a5d15f6dc7,ece16e15b2286f8287e8935f12b6c2,806c0ed32e135afb8db4f3aad71acf,f10b4adf1c732b6b260791fb24f430
2,80870ec50515107ba7034ab758cebe,cd522afebcf91625c3cd811f3fb6e2,061a59839837a123a916a70947f075,07e873c5aac8ffaf97897fbd0dfbf6,60fb638ff7ab469be8637111fdc837,d49c4a9b748b9c7abb9d98cdecebef,b0b062dc25c4dada316183941b100c,c7f809aab571e9acdf799a78c5ef61,67c38de267396951227897efc503e1,7b457a5d30f3cb01b6998390ec7abf
3,d66440e7524c0a8b9b206fbb7295ba,046f93c91d0befd4625f00e07eb473,199cf6d89a2f06fc552ebf86105d7d,2adfc14bbc0fba5b5d616d6fcfe3de,a458a5a362af9a7774780eb4e25365,88bbc5912f946b1fb77fdeaa3f5d4a,6d0bb6959972ac92c10d7445982a94,274869b912b256df69b9e7c5c0cb11,2c5bf20534b80ee47bed9a90b405a0,f456635bb3d2567a3a61133fa9dd89
4,231b80c884b2f1390e17f09a187889,bd6707bb04cfb8880a5f783c85be20,bbd5df907c099d2cc2b5285b08038e,514f3ee1f00df05dc3900251972245,961c336b0c2fa04c883c639dfee1ba,d6159553e41857fe23df9d3e746284,d2cd45a49d53702df403cffddd53e6,c0314a446fbef65ce9626b037a70da,d6e8f6d21631149a90ac940d3ee227,e762f208419b7449c0731037262bb3
5,d50faca942c93b02863dc4cc86a839,2530d7a6ecf471f695a2ef7a8a4191,9914764b34cff2aac25a82a33d5494,914880359e59dbe7b7e60c4ddc82f4,6fcff414343456465369b05f6851eb,41b772a5b18bc724562bfb60f4f3ac,34cfd44ad5cc2d63441ebf437eccb5,e291f8ae1de4a38c71027e9c91d51c,5b150a1cae009459493bd0fe5c7b3e,02fbcf93a730c9c9d5e65aef41c67e
6,5bc1c8052fb210cff1aabade8dec0c,3eb635e807cc6a8484531b00486f71,8c7ff3370563e8ad6e2fb534a1ede7,70a8a9dce94fa4e57d84c2ee735b7f,7d20e50d5f9a014a6078c92c5f17ad,05c92546aa14ea6148088b7c713be4,ff72d38f6672324e852d2cd1e089b7,e2791858e10f8f17bbb8ceb8d7e86b,d4e75ec0b19bc86bc148d32094633e,159a6475dde4d3afbb6447326500f8
7,5b150a1cae009459493bd0fe5c7b3e,e291f8ae1de4a38c71027e9c91d51c,34cfd44ad5cc2d63441ebf437eccb5,02fbcf93a730c9c9d5e65aef41c67e,26b3f811a6b9ed75d3cc5198105fa4,6fcff414343456465369b05f6851eb,e2f15819192cded77ed1825a129c0c,41b772a5b18bc724562bfb60f4f3ac,ee17bd77e91e5fb88c5610d0551d49,e3e5642e76f638dd918e718dfefe3a
8,2adfc14bbc0fba5b5d616d6fcfe3de,046f93c91d0befd4625f00e07eb473,a458a5a362af9a7774780eb4e25365,88bbc5912f946b1fb77fdeaa3f5d4a,6d0bb6959972ac92c10d7445982a94,274869b912b256df69b9e7c5c0cb11,2c5bf20534b80ee47bed9a90b405a0,f456635bb3d2567a3a61133fa9dd89,294b1604461720bdf36de6d46d96b3,227e9faf3bd31e0f87eb4afd72a69d
9,e291f8ae1de4a38c71027e9c91d51c,5b150a1cae009459493bd0fe5c7b3e,34cfd44ad5cc2d63441ebf437eccb5,02fbcf93a730c9c9d5e65aef41c67e,26b3f811a6b9ed75d3cc5198105fa4,6fcff414343456465369b05f6851eb,e2f15819192cded77ed1825a129c0c,41b772a5b18bc724562bfb60f4f3ac,ee17bd77e91e5fb88c5610d0551d49,e3e5642e76f638dd918e718dfefe3a


In [68]:
full_session_data = {}
for i in range(0, len(trailers_df)):
    unnamed_values = []
#     unnamed_values.append(trailers_df.iloc[i]['title_x'])
    unnamed_values.append(trailers_df.iloc[i]['imdb_rating'])
    unnamed_values.append(json.loads(trailers_df.iloc[i]['emotions'].replace("\'", "\""))['negative'])
    unnamed_values.append(json.loads(trailers_df.iloc[i]['emotions'].replace("\'", "\""))['positive'])
    unnamed_values.append(json.loads(trailers_df.iloc[i]['emotions'].replace("\'", "\""))['neutral'])
#     unnamed_values.append(json.loads(trailers_df.iloc[i]['Place of act'].replace("\'", "\""))[0])
#     unnamed_values.append(json.loads(trailers_df.iloc[i]['Qualities'].replace("\'", "\""))[0])
#     unnamed_values.append(json.loads(trailers_df.iloc[i]['Time of act'].replace("\'", "\""))[0])
#     unnamed_values.append(json.loads(trailers_df.iloc[i]['Based on'].replace("\'", "\""))[0])
#     unnamed_values.append(json.loads(trailers_df.iloc[i]['Mood'].replace("\'", "\""))[0])
#     unnamed_values.append(json.loads(trailers_df.iloc[i]['Subgenre'].replace("\'", "\""))[0])
#     unnamed_values.append(json.loads(trailers_df.iloc[i]['About'].replace("\'", "\""))[0])
#     unnamed_values.append(json.loads(trailers_df.iloc[i]['Theme'].replace("\'", "\""))[0])
    
    full_session_data[trailers_df.iloc[i]['trailers_name']] = unnamed_values

In [333]:
print(full_session_data['403a42b604f504069c41d39c5a4fc2'])
print(len(full_session_data))

[5.4, 0.1778208613395691, 0.051855459809303284, 0.5312193632125854]
977


In [70]:
# данные по сессиям с информацией о фильмах
session_data = []
numpy_sessions = sessions_df.to_numpy()
for i in range(0, len(numpy_sessions)):
    col = []
    for j in range(0, len(numpy_sessions[i])):
        col.append(full_session_data[numpy_sessions[i][j]])
    session_data.append(col)

print(session_data)

[[[5.4, 0.1778208613395691, 0.051855459809303284, 0.5312193632125854], [6.0, 0.1520422399044037, 0.04743587225675583, 0.5621865391731262], [8.4, 0.19194278120994568, 0.08036746829748154, 0.5775054097175598], [6.2, 0.1778208613395691, 0.04743587225675583, 0.59267657995224], [6.3, 0.19194278120994568, 0.06009664759039879, 0.6150978803634644], [5.4, 0.16027602553367615, 0.05034062638878822, 0.607673168182373], [4.3, 0.1732981950044632, 0.024433093145489693, 0.607673168182373], [7.5, 0.16886694729328156, 0.061885979026556015, 0.6442351341247559], [5.4, 0.14415885508060455, 0.09010298550128937, 0.607673168182373], [5.6, 0.18714269995689392, 0.04469086229801178, 0.6297846436500549]], [[2.9, 0.22271016240119934, 0.3140605390071869, 0.1732981950044632], [nan, 0.23371636867523193, 0.3415925204753876, 0.20182321965694427], [6.8, 0.20182321965694427, 0.320831298828125, 0.15611489117145538], [nan, 0.1824355274438858, 0.3276783227920532, 0.14805719256401062], [4.9, 0.1732981950044632, 0.29422497749

In [71]:
print(np.array(session_data).shape)

(51, 10, 4)


In [72]:
def preprocess_videoinfo(info_df):
    values = []
    values.append(info_df['temperature'].to_list())
    values.append(info_df['brightness'].to_list())
    values.append(info_df['colorfulness'].to_list())
    values.append(info_df['energy'].to_list())
    values.append(info_df['tempo'].to_list())
    values.append(info_df['amplitude'].to_list())
    values.append(info_df['spectral_rolloff'].to_list())
    values.append(info_df['mfcc'].to_list())
        
    return values
        

In [73]:
info = {}
scenes_array = []
min_len = 100
max_len = 0
del_array_name = []
info_folder = "/Volumes/Seagate/natasha-diploma/videoinfo"
for i in range(0, len(trailers_df)):
    name = trailers_df.iloc[i]['trailers_name']
    try:
        info_csv = info_folder + '/' + name + '.csv'
        info_df =  pd.read_csv(info_csv, index_col=None, header=0)
        info_df = info_df.iloc[: , 4:]
        info[name] = preprocess_videoinfo(info_df)
        scenes_array.append(preprocess_videoinfo(info_df))
    except:
        del_array_name.append(name)

In [74]:
print(info['403a42b604f504069c41d39c5a4fc2'])

[[6500.74204318, 6500.74204318, 6500.74204318, 5505.0544959, 6042.00436725, 6073.69356217, 4263.13061733, 4340.69422557, 4314.28746072, 6380.00112453, 5835.91255227, 4041.70713582, 5713.5069134, 8916.13247703, 6500.74204318, 5328.54912171, 6160.62614889, 6731.21624082, 4706.82322202, 4756.2227536, 5264.32563764, 2838.70183154, 5963.48129165, 6663.41051852, 4816.85905535, 4783.65254016, 4633.47725089, 5302.73889619, 4273.91621782, 3990.4803798, 1.03614546818e+130, 4.2303761462e+95, 1.5209842033e+89, 6500.74204318, 6500.74204318, 6500.74204318], [0.0, 0.0, 0.0, 50.2151870362, 49.2347865772, 48.7455696769, 34.2738468907, 30.6477635309, 36.6890748993, 45.380147905, 41.2990551118, 33.7454626573, 41.011673899, 18.4272309994, 0.0, 23.2737503819, 23.2854191599, 47.8570659398, 18.332016456, 34.3007160043, 55.371906235, 39.3684132364, 33.8940096992, 34.1351852216, 24.9595326351, 32.5264428578, 18.7996639777, 35.4039297418, 45.7791233874, 32.5793989662, 47.5110273349, 54.2959531071, 53.7007938704

In [75]:
# данные по сценам для каждого из трейлера
scenes_data = []
numpy_sessions = sessions_df.to_numpy()
for i in range(0, len(numpy_sessions)):
    col = []
    for j in range(0, len(numpy_sessions[i])):
        col.append(info[numpy_sessions[i][j]])
    scenes_data.append(col)

In [76]:
print(scenes_data[0][0]) # для 1 сессии 1 трейлер

[[6500.74204318, 6500.74204318, 6500.74204318, 5505.0544959, 6042.00436725, 6073.69356217, 4263.13061733, 4340.69422557, 4314.28746072, 6380.00112453, 5835.91255227, 4041.70713582, 5713.5069134, 8916.13247703, 6500.74204318, 5328.54912171, 6160.62614889, 6731.21624082, 4706.82322202, 4756.2227536, 5264.32563764, 2838.70183154, 5963.48129165, 6663.41051852, 4816.85905535, 4783.65254016, 4633.47725089, 5302.73889619, 4273.91621782, 3990.4803798, 1.03614546818e+130, 4.2303761462e+95, 1.5209842033e+89, 6500.74204318, 6500.74204318, 6500.74204318], [0.0, 0.0, 0.0, 50.2151870362, 49.2347865772, 48.7455696769, 34.2738468907, 30.6477635309, 36.6890748993, 45.380147905, 41.2990551118, 33.7454626573, 41.011673899, 18.4272309994, 0.0, 23.2737503819, 23.2854191599, 47.8570659398, 18.332016456, 34.3007160043, 55.371906235, 39.3684132364, 33.8940096992, 34.1351852216, 24.9595326351, 32.5264428578, 18.7996639777, 35.4039297418, 45.7791233874, 32.5793989662, 47.5110273349, 54.2959531071, 53.7007938704

In [77]:
print(np.array(scenes_data).shape)

(51, 10, 8)


<ipython-input-77-9e69435e55a6>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  print(np.array(scenes_data).shape)


In [78]:
train = session_data[:30]
test = session_data[30:]

input_train = []
output_train = []
for i in range(len(train)):
    input_train.append(train[i][:5])
    output_train.append(train[i][5:])
    
print(np.array(input_train).shape)

(30, 5, 4)


In [79]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=None,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True, char_level=False,
)

In [80]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')
    return tensor, lang_tokenizer

In [81]:
vocab = []
def getVocab(smth):
    for i in range(len(smth)):
        for j in range(len(smth[i])):
            for x in range(len(smth[i][j])):
                if isinstance(smth[i][j][x], str) and smth[i][j][x] not in vocab:
                    vocab.append(smth[i][j][x])



new_input_train = getVocab(input_train)
new_output_train = getVocab(output_train)

print(vocab)

[]


In [82]:
def flatten(l): return flatten(l[0]) + (flatten(l[1:]) if len(l) > 1 else []) if type(l) is list else [l]

def vocab_creater(arr):
    flat_arr = flatten(arr)
    dic = list(dict.fromkeys(flat_arr))
    word2idx = {}
    idx2word = {}
    for i in range(len(dic)):
        word2idx[dic[i]] = i
        idx2word[i] = dic[i]

    return word2idx, idx2word, len(dic)

In [83]:
word2idx, idx2word, _ = vocab_creater(vocab)

IndexError: list index out of range

In [ ]:
vocab = []
def toArray(smth):
    new = []
    
    for i in range(len(smth)):
        i_new = []
        for j in range(len(smth[i])):
            j_new = []
            for x in range(len(smth[i][j])):
                if isinstance(smth[i][j][x], str):
                    j_new.append(float(word2idx[smth[i][j][x]]))
                else:
                    j_new.append(float(smth[i][j][x]))
            i_new.append(j_new)
        new.append(i_new)
    
    return new


new_input_train = toArray(input_train)
new_output_train = toArray(output_train)

In [ ]:
print(np.array(new_input_train).shape)


In [ ]:
array_scenes = []
for i in range(len(scenes_array)):
    j_scenes = []
    for j in range(len(scenes_array[i])):
        k_scenes = []
        for k in range(len(scenes_array[i][j])):
            if scenes_array[i][j][k] == float('inf'):
                k_scenes.append(float(0))
            else:
                if (scenes_array[i][j][k] < 10000):
                    k_scenes.append(float(round(scenes_array[i][j][k])))
                else:
                    k_scenes.append(float(10000))
        j_scenes.append(k_scenes)
    array_scenes.append(j_scenes)

In [ ]:
print(np.array(array_scenes).shape)

In [ ]:
batch_size = 64
np.random.seed(1)
sessions_train = new_input_train
sessions_train_output = new_output_train
scenes_train = []
for i in range(len(sessions_df)):
    scenes_train_i = []
    for j in range(len(sessions_df.iloc[i])):
        scenes_train_i.append(info[sessions_df.iloc[i][j]])
    scenes_train.append(scenes_train_i)
        
print(np.array(scenes_train).shape)

In [84]:
def pad_nested_arrays(
    seq, maxlen_sent, maxlen_doc, pad_sent_first=True, pad_doc_first=False, pad_idx=1
):
    new_seq = []
    for i in range(len(seq)):
        j_seq = []
        for j in range(len(seq[i])):
            size = len(seq[i][j])
            if (maxlen_sent == size):
                j_seq.append(np.pad(seq[i][j], 0, 'mean'))
            else:
                size_beg = (maxlen_sent - size) // 2
                size_end = size_beg + 1
                j_seq.append(np.pad(seq[i][j], (size_beg, size_end), 'mean')[:396])
        new_seq.append(j_seq)
        
    return new_seq

In [85]:
array_scenes = []
for i in range(len(scenes_train)):
    j_scenes = []
    for j in range(len(scenes_train[i])):
        l_scenes = []
        for l in range(len(scenes_train[i][j])):
            k_scenes = []
            for k in range(len(scenes_train[i][j][l])):
                if scenes_train[i][j][l][k] == float('inf'):
                    k_scenes.append(float(0))
                else:
                    if (scenes_train[i][j][l][k] < 10000):
                        k_scenes.append(float(round(scenes_train[i][j][l][k])))
                    else:
                        k_scenes.append(float(10000))
            l_scenes.append(k_scenes)
        j_scenes.append(l_scenes)
    array_scenes.append(j_scenes)
    

train_array_scenes = array_scenes[:30]
test_array_scenes = array_scenes[30:]
print(np.array(train_array_scenes).shape)

(30, 10, 8)


<ipython-input-85-db5a6ed0ee50>:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  print(np.array(train_array_scenes).shape)


In [86]:
padded_train = np.stack([pad_nested_arrays(r, 396, 8) for r in train_array_scenes], axis=0)
print(padded_train[0][0])

[[5920.77777778 5920.77777778 5920.77777778 ... 5920.77777778
  5920.77777778 5920.77777778]
 [  30.11111111   30.11111111   30.11111111 ...   30.11111111
    30.11111111   30.11111111]
 [  30.55555556   30.55555556   30.55555556 ...   30.55555556
    30.55555556   30.55555556]
 ...
 [ -26.5         -26.5         -26.5        ...  -26.5
   -26.5         -26.5       ]
 [4098.25       4098.25       4098.25       ... 4098.25
  4098.25       4098.25      ]
 [ -12.83333333  -12.83333333  -12.83333333 ...  -12.83333333
   -12.83333333  -12.83333333]]


In [87]:
scaler = MinMaxScaler((0, 1))
normalized = []
for i in range(len(padded_train)):
    j_norm = []
    for j in range(len(padded_train[i])):
        j_norm.append(scaler.fit_transform(padded_train[i][j]))
    normalized.append(j_norm)

new = []
for i in range(len(normalized)):
    for j in range(len(normalized[i])):
        new.append(normalized[i][j])
        
normalized = np.transpose(new, (0, 2, 1))
print(np.array(normalized).shape)

(300, 396, 8)


In [96]:
sessions = []
for i in range(len(session_data)):
    for j in range(len(session_data[i])):
        sessions.append(session_data[i][j])
sessions = scaler.fit_transform(sessions)

In [97]:
trailers_dataset = tf.data.Dataset.from_tensor_slices(normalized)
info_dataset = tf.data.Dataset.from_tensor_slices(sessions)

In [337]:
seq_length = 10
examples_per_epoch = len(normalized)//(seq_length+1)

In [338]:
sequences = trailers_dataset.batch(seq_length+1, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
for seq in sequences.take(1):
    print(seq)

tf.Tensor(
[[[1.         0.00951883 0.00959356 ... 0.         0.69355261 0.00229797]
  [1.         0.00951883 0.00959356 ... 0.         0.69355261 0.00229797]
  [1.         0.00951883 0.00959356 ... 0.         0.69355261 0.00229797]
  ...
  [1.         0.00951883 0.00959356 ... 0.         0.69355261 0.00229797]
  [1.         0.00951883 0.00959356 ... 0.         0.69355261 0.00229797]
  [1.         0.00951883 0.00959356 ... 0.         0.69355261 0.00229797]]

 [[1.         0.00835881 0.00772824 ... 0.         0.55072912 0.00263501]
  [1.         0.00835881 0.00772824 ... 0.         0.55072912 0.00263501]
  [1.         0.00835881 0.00772824 ... 0.         0.55072912 0.00263501]
  ...
  [1.         0.00835881 0.00772824 ... 0.         0.55072912 0.00263501]
  [1.         0.00835881 0.00772824 ... 0.         0.55072912 0.00263501]
  [1.         0.00835881 0.00772824 ... 0.         0.55072912 0.00263501]]

 [[1.         0.00698582 0.00694235 ... 0.         0.61647253 0.00259217]
  [1.      

In [339]:
ses_sequences = info_dataset.batch(seq_length+1, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
for seq in ses_sequences.take(1):
    print(seq)

tf.Tensor(
[[0.46428571 0.34763969 0.13775814 0.53698475]
 [0.57142857 0.28482395 0.12460568 0.57368251]
 [1.         0.38205111 0.22260833 0.5918362 ]
 [0.60714286 0.34763969 0.12460568 0.60981485]
 [0.625      0.38205111 0.16228347 0.63638529]
 [0.46428571 0.30488753 0.13325007 0.62758661]
 [0.26785714 0.33661914 0.05615065 0.62758661]
 [0.83928571 0.32582135 0.16760842 0.6709145 ]
 [0.46428571 0.26561421 0.25158071 0.62758661]
 [0.5        0.37035457 0.11643668 0.65378989]
 [0.01785714 0.45702316 0.91806631 0.11282899]], shape=(11, 4), dtype=float64)


In [340]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [341]:
dataset = sequences.map(split_input_target)
sess = ses_sequences.map(split_input_target)

In [342]:
for input_example, target_example in dataset.take(1):
    print("Input :", input_example.shape)
    print("Target:", target_example.shape)
    
    
for input_example, target_example in sess.take(1):
    print("Input :", input_example.shape)
    print("Target:", target_example.shape)

Input : (10, 396, 8)
Target: (10, 396, 8)
Input : (10, 4)
Target: (10, 4)


In [343]:
scenes_size = 396
scenes_fields_size = 8
trailers_size = 5
sess_size = 4

In [344]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__(self)
        # на вход все трейлеры и на выход тоже
        self.embedding = tf.keras.layers.Embedding(977, 1024)
        self.dense = tf.keras.layers.Dense(977, activation=tf.nn.softmax)
        
    def scene_encoder(self, scenes):
        w = tf.keras.layers.LSTMCell(8)
        self.scene_lstm = tf.keras.layers.RNN(w, return_state=True)
        output = self.scene_lstm(scenes)
        
        return output
    
    def trailer_encoder(self, trailers, initial_state):
        print(trailers.shape)
        print(initial_state.shape)
        self.lstm = tf.keras.layers.LSTM(5, return_sequences=True)
        print(self.lstm.get_initial_state(trailers))
        output = self.lstm(trailers, initial_state=[initial_state[0], initial_state[1]])

        return output
    
    def call(self, trailers):
        scenes_out_s = []
        scenes_out_f = []
        for i in range(len(trailers)):
            x = self.embedding(trailers[i])
            x = tf.transpose(x, perm=[0, 2, 1])
            x = self.scene_encoder(x)
            scenes_out_s.append(x[1])
            scenes_out_f.append(x[2])
            
        input_trailers = tf.convert_to_tensor([scenes_out_s, scenes_out_f])
        output = self.trailer_encoder(trailers, input_trailers)
        output = self.dense(output)
 
        return output

In [331]:
model = MyModel(
    scenes_size=scenes_size,
    trailers_size=trailers_size,
    sess_size=sess_size)

In [332]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(">>>")

(5, 396, 8)
(2, 5, 396, 8)
[<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)>, <tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)>]


ValueError: An `initial_state` was passed that is not compatible with `cell.state_size`. Received `state_spec`=ListWrapper([InputSpec(shape=(5, 396, 8), ndim=3), InputSpec(shape=(5, 396, 8), ndim=3)]); however `cell.state_size` is [5, 5]

In [ ]:
model.summary()

In [176]:
loss = tf.losses.CategoricalCrossentropy(from_logits=True)

In [177]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [178]:
EPOCHS = 5
tf.config.run_functions_eagerly(True)

In [179]:
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/5
150/150 [==============================] - 56s 370ms/step - loss: 3.4329 - accuracy: 0.4472
Epoch 2/5
150/150 [==============================] - 55s 366ms/step - loss: 3.3819 - accuracy: 0.5620
Epoch 3/5
150/150 [==============================] - 55s 367ms/step - loss: 3.3237 - accuracy: 0.8323
Epoch 4/5
150/150 [==============================] - 58s 385ms/step - loss: 3.2622 - accuracy: 0.9097
Epoch 5/5
150/150 [==============================] - 55s 368ms/step - loss: 3.2017 - accuracy: 0.9640


In [193]:
k = 0
for input_example_batch, target_example_batch in dataset:
    t1 = input_example_batch
    t2 = target_example_batch
    model.predict(x=t1)
    if np.array_equal(t1.numpy(),t2.numpy()):
        k+=1

In [194]:
print(k)

0
